In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 15.5 MB/s eta 0:00:00


In [2]:
!pip install deep_sort_realtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.4 MB/s eta 0:00:00


In [3]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

# Load YOLO model
model = YOLO('yolov8n.pt')  # Load YOLOv8 pre-trained model

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=2, nn_budget=100)

# Load the video file
video_path = '/content/drive/MyDrive/detection_tracking_counting/video_6.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_path = '/content/drive/MyDrive/detection_tracking_counting/output_person_motorcycle.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():  # Process the entire video
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection using YOLO
    results = model(frame)
    detections = []

    for result in results:
        for box in result.boxes.data:  # Extract bounding box info
            x1, y1, x2, y2, conf, cls = map(float, box[:6])
            cls = int(cls)  # Class ID

            # Track only person (class ID 0) and motorcycle (class ID 3)
            if cls in [0, 3]:  # 0: person, 3: motorcycle
                detections.append([[x1, y1, x2, y2], conf])  # Format for DeepSORT

    # Update tracker with detections
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        x1, y1, x2, y2 = map(int, track.to_ltwh())
        track_id = track.track_id

        # Draw bounding boxes and track ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)  # Green box
        label = f'ID {track_id}'
        cv2.putText(frame, label, (x1, y1 - 15), cv2.FONT_HERSHEY_SIMPLEX,
                    1.0, (255, 0, 0), 3)  # Blue text, larger font size

    # Write the frame to output
    out.write(frame)

cap.release()
out.release()

print("Detection and tracking complete. Output saved to:", output_path)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 76.1MB/s]



0: 384x640 9 persons, 9 cars, 3 motorcycles, 1 stop sign, 1 backpack, 219.5ms
Speed: 7.3ms preprocess, 219.5ms inference, 34.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 9 cars, 4 motorcycles, 1 stop sign, 1 backpack, 149.4ms
Speed: 5.8ms preprocess, 149.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 9 cars, 4 motorcycles, 1 stop sign, 1 backpack, 149.1ms
Speed: 10.7ms preprocess, 149.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 6 cars, 4 motorcycles, 1 stop sign, 1 backpack, 221.4ms
Speed: 4.9ms preprocess, 221.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 6 cars, 4 motorcycles, 1 backpack, 224.5ms
Speed: 5.1ms preprocess, 224.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 5 cars, 4 motorcycles, 1 backpack, 230.9ms
Speed: 3.7ms preprocess, 230.9ms inference, 1.7ms postpr